In [1]:
import math
import numpy
import random
import operator
import apgl.graph
from apgl.graph import *
from apgl.generator.KroneckerGenerator import KroneckerGenerator
class Node:
    def __init__(self):
        self.loc = None
        self.time = None         #parameter to record time when this node gets hit
        self.contagion = None      #parameter to record contagion which hits this node
        
class Cascade1:
    def __init__(self):
        self.start_node = None
        self.curr_node = None
        self.cas_list = []
        
class Synthetic_Data:
    def __init__(self):
        initialGraph = SparseGraph(VertexList(5, 1))
        initialGraph.addEdge(1, 0)
        initialGraph.addEdge(2, 3)
        initialGraph.addEdge(3, 4)
        initialGraph.addEdge(1, 4)
        initialGraph.addEdge(1, 2)
        initialGraph.addEdge(1, 1)
        initialGraph.addEdge(2, 4)
        initialGraph.addEdge(3, 2)
        initialGraph.addEdge(4, 1)
        initialGraph.addEdge(4, 2)
        initialGraph.addEdge(4, 0)

        for i in range(5):
                initialGraph.addEdge(i, i)

        k = 2
        generator = KroneckerGenerator(initialGraph, k)
        self.graph = generator.generate()
        
    def print_ground_graph(self):
        print self.graph.getAllEdges()
        
    def cascade_time_generation(self, node):
#         print "Cascade time generation"
        #we choose exponential model as our incubation time model
        #parameters required
        alpha=10;
        beta=1.0;
        delta=10e-26; #not sure about this paramter, perhaps it is used to set the threshold with which we want to compare probability
        # we randomly assign time to all nodes of a network, for simplicity we select time between 0 to 60 minutes
        self.d = dict() ##dictionary to store time for each node
        random.seed(1)
        for loop_node in self.graph.getAllVertexIds():
            self.d[loop_node]=random.randint(0,60)
#             print self.d[loop_node]

        
        # now depending upon original graph, time stamps and probability law, we will propagate a cascade 
        # get immediate edges from current node
        C=Cascade1()
        C.start_node = node
        C.curr_node = node
        C.cas_list.append(C.curr_node)
        while(True):
            # find all the nodes adjacent to current location of contagion
            # find all other nodes connected to it and their corresponding distance from start_node
            temp_probs = dict()
            for j in self.graph.getAllEdges():
                if(j[0]==C.curr_node and j[1]!=C.curr_node):
                    if(self.d[j[1]]>self.d[C.curr_node]):    ##check if the neighbouring node has time greater than current node
                        # find the probability that contagion will transfer from current node to neighbouring node
                        # exponential model
                        time_diff = self.d[j[1]]-self.d[C.curr_node]
                        prob_1 = math.exp(-1.0*(time_diff/alpha))
                        prob_2 = prob_1 * beta;
#                         print "prob: "+str(prob_2)
#                         print "prob: "+str(prob_2)
                        if(prob_2>delta):      #if probability is greater than delta than append the cascade
                            temp_probs[j[1]]=prob_2
            try:
                arg_max_prob = max(temp_probs.iteritems(), key=operator.itemgetter(1))[0]
            except:
                break   
            print arg_max_prob
            C.cas_list.append(arg_max_prob)
            C.curr_node = arg_max_prob
        temp_return_list = []
        for loop_node in self.graph.getAllVertexIds():
            if(loop_node not in C.cas_list):
                temp_return_list.append('inf')
            else:
                temp_return_list.append(self.d[loop_node])
        return temp_return_list
#pass underlying graph and starting point of cascasde; and return we should get a cascade time stamp
syn_data = Synthetic_Data()
c1=Synthetic_Data.cascade_time_generation(syn_data, 21)
print c1

6
2
1
['inf', 51, 46, 'inf', 'inf', 'inf', 39, 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 33, 'inf', 'inf', 'inf']


/Users/rahuldhakecha/anaconda/lib/python2.7/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [2]:
import math
class Graph:                 #used to store graph in form of adjacency list
    def __init__(self, v):
        self.vertices = v
        self.v_arr = []
        self.graph = []     #default dictionary to store weighted edges of graph
        for i in xrange(0,self.vertices):
            self.v_arr.append([i])
                
    def add_node_vertex_list(self, n1, n2):
        self.v_arr[n1].append(n2)
      
    # function to add an edge to graph
    def addEdge(self,u,v,w):
        self.graph.append([u,v,w])
        
    def print_list_of_nodes(self, n1):
        for i in self.v_arr[n1]:
            print i,
        print 
            
    def print_graph(self):
        for i in self.v_arr:
            print "Adjaceny list of node: "+str(i[0])
            for j in i:
                print j 
                
    def max_weight_span_tree_from_DAG(self):
        T = []         #minimum weight spanning tree
        for i in xrange(0, self.vertices):
            try:
                arg_max = max([j[2] for j in self.graph if j[1] is i])
            except:
                continue
            partition = [j[0] for j in self.graph if j[1] is i and j[2] is arg_max]
            T.append([partition[0],i])
        return T
            
class Cascade:
    def __init__(self, time_stamps):
        self.n_vertices = len(time_stamps)
        self.time_stamp = []
        for i in xrange(0,self.n_vertices):
            self.time_stamp.append(time_stamps[i])
    def print_Cascade(self):
        for i in xrange(0,self.n_vertices):
            print self.time_stamp[i],
        print ""
            
    def create_DAG_from_cascade(self):
        g=Graph(self.n_vertices)
        alpha=10
        for i in xrange(0,self.n_vertices):
            for j in xrange(0,self.n_vertices):
                if(i!=j and self.time_stamp[i]!='inf' and self.time_stamp[j]!='inf' and self.time_stamp[j]>self.time_stamp[i]):
                    time_diff = self.time_stamp[j]-self.time_stamp[i]
                    prob_1 = math.exp(-1.0*(time_diff/alpha))
                    g.addEdge(i,j,prob_1)
        return g
    
    
     
if __name__ == '__main__':
    print "Hello"
#     g=Graph(5)
#     print g.vertices
#     g.add_node_vertex_list(1, 3)
#     g.print_list_of_nodes(1) 
#     g.print_list_of_nodes(3)
#     g.print_graph()
    c1 = Cascade(['inf', 51, 46, 'inf', 'inf', 'inf', 39, 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 33, 'inf', 'inf', 'inf'])
    c2 = Cascade([1,'inf',1,'inf',4])    
#     c1.print_Cascade()
#     c2.print_Cascade()
    g_ret=c1.create_DAG_from_cascade()
    print g_ret.max_weight_span_tree_from_DAG()
        

Hello
[[2, 1], [6, 2], [21, 6]]


In [ ]:
import random
import numpy as np
from numpy import unravel_index
## The NETINF Algorithm
alpha=1.0
def weight_of_edge(a,b,c):
    if(c[b]!='inf' and c[a]!='inf' and c[b]>c[a]):
        time_diff = c[b]-c[a]
        prob_1 = math.exp(-1.0*(time_diff/alpha))
        return prob_1
    else:
        return 0
    
def max_weight_receiving_node(a_node,cas):
    temp_prob = []
    for n in xrange(0,len(cas)):
        if(cas[n]!='inf' and cas[a_node]!='inf' and cas[n]>cas[a_node]):
            time_diff = cas[n]-cas[a_node]
            prob_1 = math.exp(-1.0*(time_diff/alpha))
            temp_prob.append(prob_1)
        else:
            temp_prob.append(0)
    return max(temp_prob)
    
### Part 1: Get pre-defined numbers of cascade
cascade_list = []
cascade_nos = 5
kronecker_graph = Synthetic_Data()
print 
for i in xrange(0,cascade_nos):
    random.seed(i)
#     print "random number: "+str(random.randint(0,cascade_nos))
    cascade_list.append(Synthetic_Data.cascade_time_generation(kronecker_graph, random.randint(0,24)))
    

### Part 2: For each cascade generated DAG, get maximum weight spanning tree
tree_list = []
for casc in cascade_list:
#     print casc
    casca=Cascade(casc)
    g_ret=casca.create_DAG_from_cascade()
    tree_list.append(g_ret.max_weight_span_tree_from_DAG())
#     print g_ret.max_weight_span_tree_from_DAG()
    

### Part 3: Generate underlying graph
k=10      #maximum number of edges in a graph
inf_graph = Graph(0)
while(inf_graph.vertices<k):
#     print "Cascade list length: "+str(len(cascade_list[0]))
    delta = np.zeros((len(cascade_list[0]),len(cascade_list[0])))
    for j in xrange(0,len(cascade_list[0])):             #for all nodes in cascade
        for i in xrange(0,len(cascade_list[0])):           #for all edges in cascade
            if [j,i] not in [[m[0],m[1]] for m in inf_graph.graph]:       #from the combination of edges, select ones which are not in graph
#                 print [j,i]
                cntr=0
                for c in cascade_list:
                    if c[j]<c[i]:                      #consider only those edges where the time stamp of next node is higher
#                         print "j,i"+str([j,i])
                        # take the tree corresponding to this cascake and find the edge to 'i' with maximum weight
#                         print "tree: "+str(tree_list[cntr])
#                         print "weight of edge: "+str(weight_of_edge(j,i,c))
                        if(weight_of_edge(j,i,c)>max_weight_receiving_node(i,c)):            #if weight of the given edge is greater than maximum weight in given tree
                            delta[j,i]=delta[j,i]+weight_of_edge(j,i,c)-max_weight_receiving_node(i,c)
#                             print "Delta: "+str(delta[j,i])
    print tuple((unravel_index(delta.argmax(), delta.shape)[0],unravel_index(delta.argmax(), delta.shape)[1]))
    inf_graph.addEdge(unravel_index(delta.argmax(), delta.shape)[0],unravel_index(delta.argmax(), delta.shape)[1],np.amax(delta))
#     inf_graph.add_node_vertex_list(unravel_index(delta.argmax(), delta.shape)[0],unravel_index(delta.argmax(), delta.shape)[1])                       
    inf_graph.vertices+=1
#actual underlying graph
Synthetic_Data.print_ground_graph(kronecker_graph)